<a href="https://colab.research.google.com/github/sanaz-mahmoudi/sanazmahmoudi/blob/main/OperationUnderNormalConditions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Operation Under Normal Conditions**

## Requirements

*   *Pyomo for optimization modeling*
*   *NumPy for numerical computations*
*   *GLPK solver for solving optimization problems*
*   *Matplotlib for plotting*
*   *Pandas for data handling*

In [ ]:
# PYOMO: Optimization modeling library
!pip install -q pyomo
import pyomo.environ as pe

# NUMPY: Numerical computing library
import numpy as np

# SOLVER SETUP
# Option 1: CBC solver (commented out, install if needed)
# !apt-get install -y coinor-cbc
# solver = pe.SolverFactory("cbc")

# Option 2: GLPK solver (currently used)
!apt-get install -y glpk-utils
solver = pe.SolverFactory("glpk")

# PLOTTING SETUP: Matplotlib and font configuration
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Download and add Times New Roman font for plots
!wget -O TimesNewRoman.ttf \
  https://github.com/justrajdeep/fonts/raw/master/Times%20New%20Roman.ttf

import matplotlib.font_manager as fm
import matplotlib as mpl

fm.fontManager.addfont('TimesNewRoman.ttf')

# Configure matplotlib to use Times New Roman by default
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['Times New Roman']

# DATA HANDLING: Pandas for structured data manipulation
import pandas as pd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

## Input Data

In [ ]:
print('=====================================')
print('           Load Demand Data          ')
print('=====================================')
# Load demand data: each list element contains [pmax] for a bus
data_dem = [[0],[0],[100]]
nn = len(data_dem)   # Number of buses

# Create a DataFrame for load demand with bus labels as index
dem = pd.DataFrame(data_dem,index=['n'+str(i) for i in range(nn)],columns=['pmax'])
print(dem)

print('=====================================')
print('              Line Data              ')
print('=====================================')
# Line data: list of [from_bus, to_bus, max_capacity]
data_lin = [[0,1,100],[0,2,100],[1,0,100]]

# Initialize a zero matrix for line capacities between buses
lin = pd.DataFrame(np.zeros((nn, nn), dtype=int), index=range(nn), columns=range(nn))

# Fill matrix symmetrically with line capacities
for from_bus, to_bus, pmax in data_lin:
    lin[from_bus][to_bus] = pmax
    lin[to_bus][from_bus] = pmax   # Symmetric capacity since undirected line
print(lin)

print('=====================================')
print('       Thermal Power Plant Data      ')
print('=====================================')
# Generator data: [bus, pmin, pmax, operating_cost]
data_gen = [[0,0,250,50]]
ng = len(data_gen)   # Number of thermal generation units

# DataFrame with generator info indexed by generator ID
gen = pd.DataFrame(data_gen,index=['g'+str(i) for i in range(ng)],
                   columns=['bus','pmin','pmax','o_cost'])
print(gen)

print('=====================================')
print('      Sulphur-Flow Battery Data      ')
print('=====================================')
# Battery storage data:
# Columns - bus, initial_energy, min_energy, max_energy, charge_min, charge_max, discharge_min, discharge_max, charge_eff, discharge_eff
data_bat = [[1,100,0,2400,0,100,0,100,0.9,0.9]]
nb = len(data_bat)   # Number of battery storage units

# DataFrame with battery info indexed by battery ID
bat = pd.DataFrame(data_bat,index=['b'+str(i) for i in range(nb)],
                   columns=['bus','eini','emin','emax','pcmin','pcmax','pdmin','pdmax','etac','etad'])
print(bat)

print('=====================================')
print('            Renewable Data           ')
print('=====================================')
# Renewable generation data: [bus, pmin, pmax, operating_cost]
data_ren = [[1,0,150,0]]
nw = len(data_ren)   # Number of renewable source units

# DataFrame with source info indexed by source ID
ren = pd.DataFrame(data_ren,index=['w'+str(i) for i in range(nw)],
                   columns=['bus','pmin','pmax','o_cost'])
print(ren)

print('=====================================')
print(' Hourly Demand and Renewable Profile ')
print('=====================================')
# Load hourly profiles from external CSV (normalized)
h_data = pd.read_csv('https://raw.githubusercontent.com/sanaz-mahmoudi/sanazmahmoudi/refs/heads/main/Hourly%20Profiles.csv')

nd = 28;  # Number of days
nh = 24;  # Number of hours

# Extract normalized electricity consumption profile and reshape
d_profile = h_data['Normalised Electricity Consumption Profile'].values.reshape((nd, nh))

# Initialize 3D array: days x hours x buses for load demand
pld = np.zeros((nd, nh, nn))
for i in range(nn):
    pld[:, :, i] = d_profile * dem['pmax'].values[i]
print('Hourly Load Demand =\n', pld)

# Extract normalized renewable profile and reshape
r_profile = h_data['Normalised Renewable Production Profile'].values.reshape((nd, nh))

# Initialize 3D array: days x hours x renewables for generation
pr = np.zeros((nd, nh, nw))
for i in range(nw):
    pr[:, :, i] = r_profile * ren['pmax'].values[i]
print('Hourly Renewable Generstion =\n', pr)

print('=====================================')
print('     PyPlots for Hourly Profiles     ')
print('=====================================')
# Reshape normalized profiles into weekly blocks (7 days x 24 hours)
re_h_ren_data = h_data['Normalised Renewable Production Profile'].values.reshape(-1, 7*24).T
re_h_dem_data = h_data['Normalised Electricity Consumption Profile'].values.reshape(-1, 7*24).T

# Convert to DataFrames with columns as weeks for plotting convenience
w_ren_data = pd.DataFrame(re_h_ren_data, columns=[f'week{i+1}' for i in range(re_h_ren_data.shape[1])])
print(w_ren_data)

w_dem_data = pd.DataFrame(re_h_dem_data, columns=[f'week{i+1}' for i in range(re_h_dem_data.shape[1])])
print(w_dem_data)

# -------------------- Plot 1: Renewable Production --------------------
colors = plt.cm.get_cmap('winter', w_ren_data.shape[1])

plt.figure(figsize=(10, 6))
x = np.arange(len(w_ren_data.index))

# Plot each week's renewable profile scaled by max renewable capacity
for i, week in enumerate(w_ren_data.columns):
    plt.plot(x, w_ren_data[week] * ren['pmax'].values, label=week, color=colors(i))

# Legend setup
legend_lines = [Line2D([0], [0], color=colors(i), lw=2) for i in range(w_ren_data.shape[1])]
legend_labels = [f'Week {i+1}' for i in range(w_ren_data.shape[1])]
plt.legend(legend_lines, legend_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=14)

# Axis ticks and labels
tick_positions = [1] + [24 * i for i in range(1, 8)]
tick_positions = [t for t in tick_positions if t <= 168]
plt.xticks(tick_positions, [str(t) for t in tick_positions], fontsize=12)
plt.yticks(fontsize=12)

plt.xlabel('Hours (h)', fontname='Times New Roman', fontsize=18)
plt.ylabel('Renewable Production (MW)', fontname='Times New Roman', fontsize=18)
plt.title('Renewable Production Profile', fontname='Times New Roman', fontsize=18)
plt.tight_layout()
plt.show()

# -------------------- Plot 2: Electricity Consumption --------------------
colors = plt.cm.get_cmap('winter', w_dem_data.shape[1])

plt.figure(figsize=(10, 6))
x = np.arange(len(w_dem_data.index))

# Plot each week's electricity consumption profile scaled by max load demand at bus 2
for i, week in enumerate(w_dem_data.columns):
    plt.plot(x, w_dem_data[week] * dem['pmax'].values[2], label=f'Week {i+1}', color=colors(i))

# Legend setup
legend_lines = [Line2D([0], [0], color=colors(i), lw=2) for i in range(w_dem_data.shape[1])]
legend_labels = [f'Week {i+1}' for i in range(w_dem_data.shape[1])]
plt.legend(legend_lines, legend_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=14)

# Axis ticks and labels
tick_positions = [1] + [24 * i for i in range(1, 8)]
tick_positions = [t for t in tick_positions if t <= 168]
plt.xticks(tick_positions, [str(t) for t in tick_positions], fontsize=12)
plt.yticks(fontsize=12)

plt.xlabel('Hours (h)', fontname='Times New Roman', fontsize=18)
plt.ylabel('Electricity Consumption (MW)', fontname='Times New Roman', fontsize=18)
plt.title('Electricity Consumption Profile', fontname='Times New Roman', fontsize=18)
plt.tight_layout()
plt.show()


## Optimization Model

In [ ]:
# Create a ConcreteModel object to build the optimization model
m = pe.ConcreteModel()

# Sets (Indices for various system components)
m.b  = pe.Set(initialize=list(range(nb)))                        # Set of Sulphur-flow batteries
m.g  = pe.Set(initialize=gen.index.tolist())                     # Set of thermal power plants
m.i  = pe.Set(initialize=list(range(nn)))                        # Set of network nodes
m.j  = pe.Set(initialize=list(range(nn)))                        # Set of pair nodes (for power flow)
m.td = pe.Set(initialize=list(range(nd)))                        # Set of time days
m.th = pe.Set(initialize=list(range(nh)))                        # Set of time hours
m.w  = pe.Set(initialize=list(range(nw)))                        # Set of renewable generation sources

# Decision variables
# Battery charging and discharging status
m.bcb = pe.Var(m.b,m.td,m.th,within=pe.Binary)                   # Charging status
m.bdb = pe.Var(m.b,m.td,m.th,within=pe.Binary)                   # Discharging status

# Battery power exchange
m.pcb = pe.Var(m.b,m.td,m.th,within=pe.NonNegativeReals)         # Charging power
m.pdb = pe.Var(m.b,m.td,m.th,within=pe.NonNegativeReals)         # Discharging power

# Demand shedding
m.uld = pe.Var(m.td,m.th,m.i,within=pe.NonNegativeReals)         # Demand shedding

# Thermal generator outputs
m.pg  = pe.Var(m.g,m.td,m.th,within=pe.NonNegativeReals)         # Power generation from thermal plants

# Power flows between nodes
m.pl  = pe.Var(m.i,m.j,m.td,m.th,within=pe.Reals)                # Line power flow between nodes

# Renewable power generation
m.pw  = pe.Var(m.w,m.td,m.th,within=pe.NonNegativeReals)         # Renewable energy production

# Objective: Minimize total cost
def obj_rule(m):
  return (sum(gen['o_cost'][g]*m.pg[g,td,th] for g in m.g for td in m.td for th in m.th) +
          sum(500*m.uld[td,th,i] for td in m.td for th in m.th for i in m.i))
m.obj = pe.Objective(rule=obj_rule)

# Constraints
# Ensure power generation and storage matches demand and flows
def bal_rule(m,i,td,th):
  return (sum(m.pg[g,td,th] for g in m.g if gen['bus'][g] == i) +
          sum(m.pw[w,td,th] for w in m.w if ren['bus'][w] == i) +
          sum(m.pdb[b,td,th] for b in m.b if bat['bus'][b] == i)) == (pld[td,th,i] - m.uld[td,th,i] +
                                                                      sum(m.pl[i,j,td,th] for j in m.j) +
                                                                      sum(m.pcb[b,td,th] for b in m.b if bat['bus'][b] == i))
m.bal = pe.Constraint(m.i, m.td, m.th, rule=bal_rule)

# -------------- Thermal Generation Constraints ----------------
# Thermal min generation limit
def min_gen_rule(m,g,td,th):
    return m.pg[g,td,th] >= gen['pmin'][g]
m.min_gen = pe.Constraint(m.g, m.td, m.th, rule=min_gen_rule)

# Thermal max generation limit
def max_gen_rule(m,g,td,th):
    return m.pg[g,td,th] <= gen['pmax'][g]
m.max_gen = pe.Constraint(m.g, m.td,  m.th, rule=max_gen_rule)

# -------------- Renewable Generation Constraint ---------------
# Renewable output limited by available profile
def max_ren_rule(m,w,td,th):
    return m.pw[w,td,th] <= pr[td,th,w]
m.max_ren = pe.Constraint(m.w, m.td, m.th, rule=max_ren_rule)

# ------------- Load Demand Shedding Constraint ---------------
# Load demand shedding limited by available profile
def ld_shed_rule(m,td,th,i):
    return m.uld[td,th,i] <= pld[td,th,i]
m.ld_shed = pe.Constraint(m.td, m.th, m.i, rule=ld_shed_rule)

# --------- Sulphur-Flow Battery Operation Constraints ---------
# Minimum charging limit
def min_cha_rule(m,b,td,th):
    return m.pcb[b,td,th] >= bat['pcmin'][b]*m.bcb[b,td,th]
m.min_cha = pe.Constraint(m.b, m.td, m.th, rule=min_cha_rule)

# Maximum charging limit
def max_cha_rule(m,b,td,th):
    return m.pcb[b,td,th] <= bat['pcmax'][b]*m.bcb[b,td,th]
m.max_cha = pe.Constraint(m.b, m.td, m.th, rule=max_cha_rule)

# Minimum discharging limit
def min_dis_rule(m,b,td,th):
    return m.pdb[b,td,th] >= bat['pdmin'][b]*m.bdb[b,td,th]
m.min_dis = pe.Constraint(m.b, m.td, m.th, rule=min_dis_rule)

# Maximum discharging limit
def max_dis_rule(m,b,td,th):
    return m.pdb[b,td,th] <= bat['pdmax'][b]*m.bdb[b,td,th]
m.max_dis = pe.Constraint(m.b, m.td, m.th, rule=max_dis_rule)

# Minimum stored energy range
def min_sto_rule(m,b,td):
    return bat['emin'][b] <= (bat['eini'][b] +
                              sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th))
m.min_sto = pe.Constraint(m.b, m.td, rule=min_sto_rule)

# Maximum stored energy range
def max_sto_rule(m,b,td):
    return bat['eini'][b] + sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th) <= bat['emax'][b]
m.max_sto = pe.Constraint(m.b, m.td, rule=max_sto_rule)

# Initial-final SOC balance (cyclic operation)
def bid_sto_rule(m,b,td):
    return sum(bat['etac'][b]*m.pcb[b,td,th] - (1/bat['etad'][b])*m.pdb[b,td,th] for th in m.th) == 0
m.bid_sto = pe.Constraint(m.b, m.td, rule=bid_sto_rule)

# Prevent simultaneous charging/discharging
def cha_dis_rule(m,b,td,th):
    return m.bcb[b,td,th] + m.bdb[b,td,th] <= 1
m.cha_dis = pe.Constraint(m.b, m.td, m.th, rule=cha_dis_rule)

# ------------- Power Flow Constraints (Lines) ---------------
# Line flow lower bound
def min_flow_rule(m,i,j,td,th):
    return m.pl[i,j,td,th] >= -lin.loc[i,j]
m.min_flow = pe.Constraint(m.i, m.j, m.td, m.th, rule=min_flow_rule)

# Line flow upper bound
def max_flow_rule(m,i,j,td,th):
    return m.pl[i,j,td,th] <= lin.loc[i,j]
m.max_flow = pe.Constraint(m.i, m.j, m.td, m.th, rule=max_flow_rule)

# Line flow symmetry (Kirchhoff’s law)
def adj_flow_rule(m,i,j,td,th):
    return m.pl[i,j,td,th] == -m.pl[j,i,td,th]
m.adj_flow = pe.Constraint(m.i,m.j,m.td,m.th,rule=adj_flow_rule)

# Solve using GLPK with a relative gap of 1e-5
solver.options['mipgap'] = 1e-5
solver.solve(m,tee=True).write()


## Print Results

In [ ]:
# Display the optimal value of the objective function
print('Optimal Value =',m.obj())

# --------------- Thermal Generation Results -----------------
# Extract and organize power output from thermal units
data = []
for (g, td, th) in m.pg:
            value = m.pg[g,td,th].value
            data.append({
                'g':g,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['g', 'td', 'th'])
thermal = df['g'].unique()

# Display hourly thermal generation for each unit
for g in thermal:
    sub_df = df[df['g'] == g]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nThermal Generation: {g}")
    display(pivot)

# ------------- Renewable Generation Results ----------------
# Extract and organize power output from renewable sources
data = []
for (w, td, th) in m.pw:
            value = m.pw[w,td,th].value
            data.append({
                'w':w,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['w', 'td', 'th'])
renewable = df['w'].unique()

# Display hourly renewable generation for each source
for w in renewable:
    sub_df = df[df['w'] == w]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nRenewable Generation: {w}")
    display(pivot)

# ------------- Load Demand Shedding Results ----------------
# Extract and organize load shedding
data = []
for (td, th, i) in m.uld:
            value = m.uld[td,th,i].value
            data.append({
                'i':i,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['i', 'td', 'th'])
shedding = df['i'].unique()

# Display hourly load demand shedding for each bus
for n in shedding:
    sub_df = df[df['i'] == i]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nDemand Shedding: {i}")
    display(pivot)

# --------------- Battery Charging Results -----------------
# Extract charging power for each battery
data = []
for (b, td, th) in m.pcb:
            value = m.pcb[b,td,th].value
            data.append({
                'b':b,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['b', 'td', 'th'])
batteries = df['b'].unique()

# Display hourly charging profiles
for b in batteries:
    sub_df = df[df['b'] == b]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nCharging Batteries: {b}")
    display(pivot)

# -------------- Battery Disharging Results ---------------
# Extract discharging power for each battery
data = []
for (b, td, th) in m.pdb:
            value = m.pdb[b,td,th].value
            data.append({
                'b':b,
                'td': td,
                'th': th,
                'value': value
                })

df = pd.DataFrame(data)
df = df.sort_values(by=['b', 'td', 'th'])
batteries = df['b'].unique()

# Display hourly discharging profiles
for b in batteries:
    sub_df = df[df['b'] == b]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    print(f"\nDischarging Batteries: {b}")
    display(pivot)

# ----------- Power Flow Between Nodes (Lines) ------------
# Extract line flows between all node pairs
data = []
for (i, j, td, th) in m.pl:
    value = m.pl[i,j,td,th].value
    data.append({
        'i': i,
        'j': j,
        'td': td,
        'th': th,
        'value': value
        })

df = pd.DataFrame(data)
df = df.sort_values(by=['i', 'j', 'td', 'th'])

# Create a unique label for each directional line
df['group'] = df['i'].astype(str) + ' → ' + df['j'].astype(str)

# Pivot each group (line) and display
pivoted_tables = {}
groups = df['group'].unique()

for group in groups:
    sub_df = df[df['group'] == group]
    pivot = sub_df.pivot(index='td', columns='th', values='value')
    pivoted_tables[group] = pivot

for group, table in pivoted_tables.items():
    print(f"\n=== {group} ===")
    display(table)


# Details of the Worst Operating Condition


> *Hour with the Highest Demand and Lowest Renewable Generation — i.e., the Highest Net Load*



In [ ]:
# Initialize a 2D numpy array pnl with zeros to store net load values
# Dimensions: number of days (nd) x number of hours (nh)
pnl = np.zeros((nd,nh))

# Loop over every day and hour to compute net load for bus 2
for td in range(nd):
  for th in range(nh):
    # Net load = total demand at bus 2 minus total renewable generation at all renewable buses
    # pld[td, th, 2]: demand at bus 2 at time (td, th)
    # pr[td, th, w]: renewable generation at renewable bus w at time (td, th)
    pnl[td,th] = pld[td,th,2] - sum(pr[td,th,w] for w in m.w)

# Trim pnl to only the first 4 weeks (assuming 7 days per week)
# pnl[:4*7, :] selects the first 28 days of data
pnl_trimmed = pnl[:4 * 7, :]

# Reshape pnl_trimmed for plotting by weeks:
# - reshape to (4 weeks, 7 days, nh hours)
# - transpose to bring days and hours together and weeks last: shape becomes (7, nh, 4)
# - reshape to combine days and hours for x-axis: shape (nh*7, 4)
pnl = pnl_trimmed.reshape(4, 7, nh).transpose(1, 2, 0).reshape(nh*7, 4)

# Set up a colormap with number of colors equal to number of weeks (4)
colors = plt.cm.get_cmap('winter', pnl.shape[1])

plt.figure(figsize=(10, 6))
x = np.arange(len(pnl))

# Plot net load profile for each week
for i, week in enumerate(pnl.T):  # Transpose to loop over weeks
    plt.plot(x, week, label=f'Week {i+1}', color=colors(i))

# Custom legend
legend_lines = [Line2D([0], [0], color=colors(i), lw=2) for i in range(pnl.shape[1])]
legend_labels = [f'Week {i+1}' for i in range(pnl.shape[1])]
plt.legend(legend_lines, legend_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=14)

# Axis ticks and labels
tick_positions = [1] + [24 * i for i in range(1, 8)]
tick_positions = [t for t in tick_positions if t <= 168]
plt.xticks(tick_positions, [str(t) for t in tick_positions], fontsize=12)
plt.yticks(fontsize=12)

plt.xlabel('Hours (h)', fontname='Times New Roman', fontsize=18)
plt.ylabel('Net Load (MW)', fontname='Times New Roman', fontsize=18)
plt.title('Net Load Profile', fontname='Times New Roman', fontsize=18)
plt.tight_layout()
plt.show()

# Find the day (td) and hour (th) indices where pnl is maximum over all days and hours
max_td, max_th = np.unravel_index(np.argmax(pnl), pnl.shape)

# Print the maximum net load value and its location (add 1 for 1-based counting)
print(f"max_pnl[{max_td+1}, {max_th+1}] = {pnl[max_td, max_th]}")

# For each generator g in the model, print the generation at the max net load time
for g_max in m.g:
    pg_max = m.pg[g_max,max_td,max_th].value  # get the numeric value of generation
    print(f"pg_max[{g_max}, {max_td+1}, {max_th+1}] = {pg_max}")

# For node 2, print the load shedding at the max net load time
uld_max = m.uld[max_td,max_th,2].value  # get the numeric value of generation
print(f"uld_max[{max_td+1}, {max_th+1}, {2}] = {uld_max}")

# For each renewable bus w, print renewable generation at max net load time
for w_max in m.w:
    pw_max = m.pw[w_max,max_td,max_th].value
    print(f"pw_max[{w_max}, {max_td+1}, {max_th+1}] = {pw_max}")

# For each battery b, print charging power at max net load time
for cb_max in m.b:
    pcb_max = m.pcb[cb_max,max_td,max_th].value
    print(f"pcb_max[{cb_max}, {max_td+1}, {max_th+1}] = {pcb_max}")

# For each battery b, print discharging power at max net load time
for db_max in m.b:
    pdb_max = m.pdb[db_max,max_td,max_th].value
    print(f"pdb_max[{db_max}, {max_td+1}, {max_th+1}] = {pdb_max}")

# For each branch i to j, print the power flow at max net load time
for i_max in m.i:
  for j_max in m.j:
    pl_max = m.pl[i_max,j_max,max_td,max_th].value
    print(f"pl_max[{i_max}, {j_max}, {max_td+1}, {max_th+1}] = {pl_max}")
